In [1]:
import wikipediaapi
from dotenv import load_dotenv
import os
# Wikipedia article setup
webpage = "Python (programming language)" # wikipedia article title
load_dotenv()

def fetchWiki(term, wiki_lang="en"):
    wiki = wikipediaapi.Wikipedia(user_agent=os.getenv('wikipedia_useragent'), language=wiki_lang)
    page = wiki.page(term)
    if page.exists():
        return page.text
        result = page.section_by_title("Plot") or page.section_by_title("Synopsis")
        return result
    else:
        print(f"{term} could not be found")
        return ""

with open('input.txt', 'r') as f, open("output.txt", "w") as w:
    for line in f:
        line = line.strip()
        w.write(f"{fetchWiki(line)}\n\n\n")

In [25]:
import pickle

with open('output.txt', 'r', encoding='utf-8') as f:
    text = f.read()
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

vocab_size = 500
num_merges = vocab_size - 256
ids = list(tokens)
merge_interval = 30

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    if (i % merge_interval == 0):
        print(f"{i} merging {pair} to {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx


vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

with open('mergedValues.txt', 'w') as f:
    for i in range(vocab_size):
        f.write(f"[{vocab[i].decode("utf-8", errors="replace")}] = {i}\n")

with open('tokened.txt', 'wb') as f:
    pickle.dump(vocab, f)

with open('merges.txt', 'wb') as f:
    pickle.dump(merges, f)

0 merging (101, 32) to 256
30 merging (97, 32) to 286
60 merging (46, 10) to 316
90 merging (97, 103) to 346
120 merging (119, 268) to 376
150 merging (111, 99) to 406
180 merging (270, 108) to 436
210 merging (115, 99) to 466
240 merging (102, 105) to 496
